# Camera Anomaly Detection V2 - Training Notebook

This notebook is designed to run on Google Colab (A100 GPU) or locally.
It uses Multiple Instance Learning (MIL) with a ResNet50V2 backbone.

In [ ]:
# 1. Setup Environment
import sys
import os

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # UPDATE THIS PATH TO MATCH YOUR DRIVE LOCATION
    PROJECT_PATH = '/content/drive/MyDrive/Colab Notebooks/camera_anomaly_detection_v2'
    
    if not os.path.exists(PROJECT_PATH):
        print(f"WARNING: Project path {PROJECT_PATH} not found. Please check your Drive structure.")
    else:
        os.chdir(PROJECT_PATH)
        print(f"Changed directory to {PROJECT_PATH}")
        
    # Install dependencies
    !python setup_env.py

In [ ]:
# 2. Verify Config and GPU
from config import config
from pathlib import Path

# Standard Configuration for Cloud/AWS
# config.DATA_DIR = Path('data/DCSASS Dataset') # Default
# LOCAL QUICK RUN CONFIG
config.EPOCHS = 1
config.BATCH_SIZE = 1
config.MAX_LABELS = None

# Monkey-patch loader to only load Normal and Abuse
import dcsass_loader
original_load_metadata = dcsass_loader._load_metadata

def restricted_load_metadata(root, seed=1337):
    entries = original_load_metadata(root, seed)
    # Filter for only Normal and Abuse
    target_labels = {'Normal', 'Abuse'}
    filtered_entries = [e for e in entries if e['label'] in target_labels]
    
    print(f"Filtered dataset to {len(filtered_entries)} samples from classes: {target_labels}")
    return filtered_entries

dcsass_loader._load_metadata = restricted_load_metadata

config.display()

import tensorflow as tf
print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

# Enable Mixed Precision for T4/A100 speedup and memory savings
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
print(f"Mixed precision policy set to: {policy.name}")

In [ ]:
# 3. Load Data
from dcsass_loader import make_bag_dataset

print("Creating datasets...")
# Note: batch_size=1 means 1 bag per batch. Each bag contains T clips.
train_ds = make_bag_dataset(
    root=None, 
    split="train", 
    batch_size=1, 
    cache_decoded=True
)
val_ds = make_bag_dataset(
    root=None, 
    split="val", 
    batch_size=1, 
    cache_decoded=True
)

print("Datasets created.")

In [ ]:
# 4. Build Model
from model import build_mil_model

model = build_mil_model()
model.summary()

In [ ]:
# 5. Training Loop
optimizer = tf.keras.optimizers.Adam(learning_rate=config.LEARNING_RATE)
bce_loss = tf.keras.losses.BinaryCrossentropy()
from tqdm import tqdm

@tf.function
def train_step(bag, label):
    # bag: (1, T, H, W, C) - RaggedTensor or Tensor
    # label: (1,)
    
    instances = bag[0] # (Num_Segments, T, H, W, C)
    
    # Flatten segments and time: (Num_Segments * T, H, W, C)
    # ResNet50 expects 4D input (Batch, H, W, C)
    flat_instances = tf.reshape(instances, (-1, 224, 224, 3))
    
    with tf.GradientTape() as tape:
        scores = model(flat_instances, training=True)
        max_score = tf.reduce_max(scores)
        
        # Ensure shapes are (1,) for BCE
        max_score = tf.expand_dims(max_score, 0)
        loss = bce_loss(label, max_score)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    return loss, max_score

print("Starting training...")
for epoch in range(config.EPOCHS):
    print(f"\nEpoch {epoch+1}/{config.EPOCHS}")
    
    # Training
    total_loss = 0.0
    steps = 0
    for bag, label, vid_id in tqdm(train_ds, desc="Training"):
        # bag is RaggedTensor, convert to tensor if needed, but bag[0] should work if it's uniform T
        # If T varies, bag[0] is a tensor of shape (T_actual, H, W, C)
        if isinstance(bag, tf.RaggedTensor):
            bag = bag.to_tensor()
            
        loss, score = train_step(bag, label)
        total_loss += loss
        steps += 1
        
    avg_loss = total_loss / steps
    print(f"Train Loss: {avg_loss:.4f}")
    
    # Validation
    val_loss = 0.0
    val_steps = 0
    correct = 0
    total = 0
    
    for bag, label, vid_id in tqdm(val_ds, desc="Validation"):
        if isinstance(bag, tf.RaggedTensor):
            bag = bag.to_tensor()
            
        instances = bag[0]
        
        # Flatten segments and time for validation as well
        flat_instances = tf.reshape(instances, (-1, 224, 224, 3))
        
        scores = model(flat_instances, training=False)
        max_score = tf.reduce_max(scores)
        
        # Ensure shapes for BCE
        max_score = tf.expand_dims(max_score, 0)
        loss = bce_loss(label, max_score)
        
        val_loss += loss
        val_steps += 1
        
        # Accuracy
        pred = 1 if max_score > 0.5 else 0
        if pred == label[0]:
            correct += 1
        total += 1
        
    if val_steps > 0:
        print(f"Val Loss: {val_loss/val_steps:.4f}, Val Acc: {correct/total:.4f}")
    else:
        print("Val Loss: N/A, Val Acc: N/A (No validation samples)")
    
    # Save Checkpoint
    if (epoch + 1) % 5 == 0:
        ckpt_path = config.CHECKPOINT_DIR / f"model_epoch_{epoch+1}.h5"
        config.CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)
        model.save_weights(str(ckpt_path))
        print(f"Saved checkpoint to {ckpt_path}")